In [1]:
import { TextLoader } from "langchain/document_loaders/fs/text";
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";
const loader = new TextLoader("data/qiu.txt");
const docs = await loader.load();

const splitter = new RecursiveCharacterTextSplitter({
    chunkSize: 100,
    chunkOverlap: 20,
  });

const splitDocs = await splitter.splitDocuments(docs);
console.log('splitDocs :', splitDocs);


splitDocs : [
  Document {
    pageContent: "大家好，我是程序员鱼皮，8 月 19 日下午，网易云音乐突发严重故障，并登顶微博热搜，跟黑神话悟空抢了热度。",
    metadata: { source: "data/qiu.txt", loc: { lines: { from: 1, to: 1 } } }
  },
  Document {
    pageContent: "根据用户的反馈，故障的具体表现为：用户无法登录、歌单加载失败、播放信息获取失败、无法搜索歌曲等等，几乎是无法使用了，妥妥的 P0 级事故！",
    metadata: { source: "data/qiu.txt", loc: { lines: { from: 3, to: 3 } } }
  },
  Document {
    pageContent: "根据官方发布的说明，本次故障的主要原因是基础设施，导致网易云音乐各端无法正常使用：",
    metadata: { source: "data/qiu.txt", loc: { lines: { from: 5, to: 5 } } }
  },
  Document {
    pageContent: "什么是基础设施？是指支持整个系统运行的基础性服务和资源，包括服务器、网络设备、数据库、存储系统、内容分发网络（CDN）、各种云服务、缓存、DNS、负载均衡等等。像之前 B",
    metadata: { source: "data/qiu.txt", loc: { lines: { from: 7, to: 7 } } }
  },
  Document {
    pageContent: "B 站和小红书大规模故障，就是因为某云服务商的网络出了问题，可见基础设施的重要性。",
    metadata: { source: "data/qiu.txt", loc: { lines: { from: 7, to: 7 } } }
  },
  Document {
    pageContent: "我不是内部人员，所以具体的故障原因不得而知，网上有很多猜测，什么 “开发删库跑路”、“搬迁到新机房产生了问题”、“裁员导致降本增笑” 等等，但这些说法被官方否认了。",


In [2]:
import { OllamaEmbeddings } from '@langchain/community/embeddings/ollama';
const embeddings = new OllamaEmbeddings({
  model: 'llama3.1', // default value
  baseUrl: 'http://localhost:11434', // default value
  requestOptions: {
    useMMap: true, // use_mmap 1
    numThread: 10, // num_thread 10
    numGpu: 1, // num_gpu 1
  },
});

const res =  await embeddings.embedQuery(splitDocs[0].pageContent)
console.log('res :', res);


res : [
   -1.3003714084625244,  -1.0172111988067627,    2.2036068439483643,
   -1.9257409572601318,   -5.177871227264404,     1.540187954902649,
   -0.4401560425758362,   1.3011869192123413,     7.371114730834961,
    1.2314167022705078,  -0.9582812786102295,   -1.0799325704574585,
    -1.039917230606079,  -0.3526543974876404,   -0.2675212621688843,
   -0.9581142067909241, -0.13173626363277435,    2.8560221195220947,
    -0.370891809463501, -0.16745661199092865,   -1.2123363018035889,
   0.30914294719696045, -0.06695081293582916,     2.571530818939209,
    -3.654949426651001,  -0.5353257060050964,    4.1314592361450195,
    1.1327871084213257,  -3.4183852672576904,   -3.4177534580230713,
   -1.5600056648254395,   1.8888368606567383,    0.8575529456138611,
   -0.6101484298706055,    2.987922191619873, -0.057151854038238525,
  0.012492441572248936,   0.6390185356140137,   0.05501732975244522,
    -3.174879312515259,  -2.2843213081359863,   -0.4062038064002991,
   -0.8799357414245605,  -

In [3]:
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const vectorstore = new MemoryVectorStore(embeddings);
await vectorstore.addDocuments(splitDocs);

const retriever = vectorstore.asRetriever(2)

const result = await retriever.invoke("事故是怎么产生的")
console.log('result :', result);


result : [
  Document {
    pageContent: "朋友们，你们怎么看待这次故障，有怀疑过自己网络或设备的问题么？",
    metadata: { source: "data/qiu.txt", loc: { lines: { from: 27, to: 27 } } }
  },
  Document {
    pageContent: "根据用户的反馈，故障的具体表现为：用户无法登录、歌单加载失败、播放信息获取失败、无法搜索歌曲等等，几乎是无法使用了，妥妥的 P0 级事故！",
    metadata: { source: "data/qiu.txt", loc: { lines: { from: 3, to: 3 } } }
  }
]
